In [ ]:
import numpy as np
import scipy.sparse as sp
from scipy.sparse.linalg import spsolve
import matplotlib.pyplot as plt

%matplotlib notebook

In [ ]:
def calc_A(x):
    N = x.size - 2
    dx = np.diff(x)
    
    row_ind = np.concatenate((
         np.arange(N),
         np.arange(N-1),
         np.arange(N-1)+1
    ))
    col_ind = np.concatenate((
         np.arange(N),
         np.arange(N-1)+1,
         np.arange(N-1)
    ))
    
    data = np.concatenate((
        (1.0 / dx[0:-1]) + (1 / dx[1:]),
        -(1 / dx[1:-1]),
        -(1 / dx[1:-1]),
    ))
    
    return sp.csr_matrix((data, (row_ind, col_ind)), shape=(N, N))


def calc_F(x):
    """assume f = 1 on Omega"""
    
    return 0.5*(x[2:]-x[:-2])
    
def calc_UX(x):
    N = x.size - 2
    dx = np.diff(x)
    
    row_ind = np.concatenate((
         np.arange(N),
         np.arange(N)+1
    ))
    col_ind = np.concatenate((
         np.arange(N),
         np.arange(N)
    ))
    
    data = np.concatenate((
        (1.0 / dx[0:-1]),
        -(1 / dx[1:]),
    ))
    return sp.csr_matrix((data, (row_ind, col_ind)), shape=(N+1, N))


def calc_U(x):
    N = x.size - 2
    
    row_ind = np.concatenate((
         np.arange(N),
         np.arange(N)+1
    ))
    col_ind = np.concatenate((
         np.arange(N),
         np.arange(N)
    ))
    
    data = 0.5*np.ones((2*N,))
    
    return sp.csr_matrix((data, (row_ind, col_ind)), shape=(N+1, N))

def calc_D(x):
    N = x.size - 2
    return sp.diags(np.diff(x))

def calc_F2(x, Ft):
    U = calc_U(x)
    D = calc_D(x)
    return U.transpose().dot(D).dot(Ft)

def rand_x(N=10):
    x = np.random.rand(N+2)
    x[0] = 0
    x[-1] = 1
    x.sort()
    return x
    
def unif_x(N=10):
    return np.linspace(0, 1, N+2)

def main1(N):
    x = np.linspace(0, 1, N+2)
    xx = x
    A = calc_A(x)
    f = calc_F(x)
    
    u = np.zeros((N+2,), dtype=np.float)
    u[1:-1] = spsolve(A, f)
    
    uu = -0.5*(xx-0.5)**2 + 1.0/8
    
    
    plt.figure()
    plt.plot(xx, uu, 'g', label='Analytic')
    plt.plot(x, u, '+-r', label='FEM solution')
    plt.legend()
    plt.show()
    
    print("N = {}, e_h = {}".format(N, np.amax(np.abs(np.interp(xx, x, u) - uu))))
    
def main2(N = 10):
    x = rand_x(N)
    xx = unif_x(100*N)
    
    UX = calc_UX(x)
    D = calc_D(x)
    A = UX.transpose().dot(D).dot(UX)
    
    Ft = 0.5*(x[0:-1] + x[1:])
    f = calc_F2(x, Ft)
    
    u = np.zeros((N+2,), dtype=np.float)
    u[1:-1] = spsolve(A, f)
    
    uu = xx * (1-xx**2)/6
    
    
    plt.figure()
    plt.plot(xx, uu, 'g', label='Analytic')
    plt.plot(x, u, '+-r', label='FEM solution')
    plt.legend()
    plt.show()
    
    print("N = {}, e_h = {}".format(N, np.amax(np.abs(np.interp(xx, x, u) - uu))))
    
def main3(epsilon=0.01, N = None):
    if N is None:
        N = 5
        x = np.array([0.0, 0.75, 0.875, 0.9375, 0.96875, 0.984375, 1.0])
    else:
        x = unif_x(N)
    
    xx = unif_x(1000)
    
    U = calc_U(x)
    UX = calc_UX(x)
    D = calc_D(x)
    A = epsilon*UX.transpose().dot(D).dot(UX) + U.transpose().dot(D).dot(UX)
    
    Ft = np.ones((N+1,))
    f = U.transpose().dot(D).dot(Ft)
    
    u = np.zeros((N+2,), dtype=np.float)
    u[1:-1] = spsolve(A, f)
    
    uu = xx - (np.expm1(xx/epsilon) /np.expm1(1/epsilon))
    
    
    plt.figure()
    plt.plot(xx, uu, 'g', label='Analytic')
    plt.plot(x, u, '+-r', label='FEM solution')
    plt.legend()
    plt.show()
    
    print("N = {}, e_h = {}".format(N, np.amax(np.abs(np.interp(xx, x, u) - uu))))

In [ ]:
main3()

In [ ]:
x = rand_x()

UX = calc_UX(x)
D = calc_D(x)
A = calc_A(x)

A2 = UX.transpose().dot(D).dot(UX)

if (A-A2).nnz == 0:
    e_A = 0
else:
    e_A = np.amax(np.abs((A-A2).data))
print("e_A = {}".format(e_A))